In [6]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

E: Package 'python-software-properties' has no installation candidate


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
cd drive

/content/drive


In [9]:
cd My Drive

/content/drive/My Drive


In [10]:
import keras
import os
os.chdir('/content/drive/My Drive/44Col/')
print(os.listdir())

['94_2', '93_2', '92_2', '98_3', '99_3', '9_0', '95_2', '91_2', '96_2', '97_3', '8_0', '88_2', '85_2', '90_2', '86_2', '83_2', '89_2', '82_2', '87_2', '84_2', '7_0', '74_2', '73_2', '80_2', '79_2', '81_2', '75_2', '77_2', '78_2', '76_2', '64_2', '72_2', '66_2', '68_2', '65_2', '71_2', '70_2', '69_2', '67_2', '6_0', '60_2', '62_2', '57_2', '61_2', '56_2', '58_2', '5_0', '55_1', '59_2', '63_2', '52_1', '53_1', '49_1', '54_1', '4_0', '48_1', '51_1', '47_1', '50_1', '46_1', '38_1', '37_1', '41_1', '43_1', '39_1', '45_1', '40_1', '3_0', '44_1', '42_1', '33_0', '35_1', '36_1', '34_0', '32_0', '2_0', '28_0', '30_0', '29_0', '31_0', '26_0', '24_0', '232_5', '22_0', '229_5', '230_5', '231_5', '27_0', '25_0', '23_0', '227_5', '228_5', '224_5', '221_5', '21_0', '226_5', '225_5', '220_5', '223_5', '222_5', '217_5', '218_5', '213_4', '219_5', '211_4', '212_4', '214_4', '216_5', '210_4', '215_5', '205_3', '209_3', '208_3', '201_3', '204_3', '207_3', '20_0', '206_3', '202_3', '203_3', '196_3', '1_0',

In [11]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [15]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  5 16:23:15 2018

@author: Pooja
"""
#Meand and Cross Validation
import numpy as np
import os
import cv2
import keras
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense, Flatten, Dropout,Input, Bidirectional,BatchNormalization
#from keras.layers import Merge
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed	
from keras.layers.convolutional import (Conv2D,MaxPooling2D)
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint	
from keras.models import load_model
from keras.layers import multiply
from keras.layers.core import *
from keras.models import Model
max_seq = 44
num = np.zeros(6)
path_uiuc = '/content/drive/My Drive/'
x = list()
y = list()
#reading color image - changin to gray and subtracting mean
data_fold = ['44D1', '44D2', '44Col']
for fold in data_fold:
    path_44 = path_uiuc + fold + '/'
    print(path_44)
    vidlist = os.listdir(path_44)
    it = len(vidlist)
    for i in range(it):
        s1 = vidlist[i]
        _, typ = s1.split('_')
        #NO REACT
        if typ == '3':
            y.append(3)
        
        elif typ == '0':
           y.append(0)
           
        elif typ == '1':
           y.append(1)
           
        elif typ == '2':
           y.append(2)
           
        elif typ == '4':   
           y.append(4)
           
        elif typ == '5':
           y.append(5)
        else:
            print('Error!' + str(typ))
            
        num[int(typ)] +=1
        
        os.chdir(path_44 + s1)   
        img_list = os.listdir()
        it0 = len(img_list)
        temp = list()
        dim = (32,32)
        for j in range(it0):
            img = cv2.imread(img_list[j])
            #print(str(i)+'_'+str(j))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
            mn1 = np.mean(img)
            st1 = np.std(img)
            img = (img.copy() - mn1) / st1
            temp.append(img)
        temp = np.asarray(temp)    
        x.append(temp)
        
strokes = {'backhand':num[0], 'forehand':num[1], 'lob':num[2], 'react':num[3], 'serve':num[4], 'smash':num[5]}
for i in strokes.keys():
    print("{} - %d".format(i)%(strokes[i]))
    
x = np.asarray(x)
y = np.asarray(y)
print(x.shape)
x = x.reshape(x.shape[0], x.shape[1], x.shape[2], x.shape[3], 1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42,shuffle =True)
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
#y_train = np_utils.to_categorical(y_train, 6)
y_test = np_utils.to_categorical(y_test, 6)

seed = 10   
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
ii = 0 

#model
x = Input((44,32,32,1))
model = (TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same')))(x)
model = (TimeDistributed(Conv2D(32, (3,3),  activation='relu')))(model)
model = (TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))(model)
model = (BatchNormalization())(model)
#model = (Dropout(0.25)(model)

model = (TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same')))(model)
model = (TimeDistributed(Conv2D(64, (3,3),  activation='relu')))(model)
model = (TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))(model)
model = (BatchNormalization())(model)
#model = (Dropout(0.25)(model)

model = (TimeDistributed(Flatten()))(model)

model = (Dropout(0.5))(model)
model = (Bidirectional(LSTM(512, return_sequences=False, dropout=0.4)))(model)
model = (Dense(6, activation='softmax'))(model)
model = Model(input = x, output = model)
print(model.summary())

for train, test in kfold.split(X_train, y_train):
    print('------------ITERATION : ' + str(ii) + '  -------------------') 
	# Compile model
    #sgd = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    sgd = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
	# Fit the model
    Yy = np_utils.to_categorical(y_train, 6)
    model.fit(X_train[train], Yy[train], epochs=80, batch_size=32, verbose=1)
    model.save('/content/drive/My Drive/conv_s_m'+str(ii)+'.h5')
    ii += 1
    #print(model.predict(X[train], batch_size=32))
	# evaluate the model
    scores = model.evaluate(X_train[test], Yy[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

final_scores = []
for i in range(5):
  mod = load_model('/content/drive/My Drive/'+'conv_all2_m'+str(i)+'.h5')
  scores = model.evaluate(X_test, y_test, verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  final_scores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(final_scores), np.std(final_scores)))

/content/drive/My Drive/44D1/
/content/drive/My Drive/44D2/
/content/drive/My Drive/44Col/
backhand - 57
forehand - 62
lob - 69
react - 196
serve - 7
smash - 36
(427, 44, 32, 32)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:124: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 44, 32, 32, 1)     0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 44, 32, 32, 32)    320       
_________________________________________________________________
time_distributed_16 (TimeDis (None, 44, 30, 30, 32)    9248      
_________________________________________________________________
time_distributed_17 (TimeDis (None, 44, 15, 15, 32)    0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 44, 15, 15, 32)    128       
_________________________________________________________________
time_distributed_18 (TimeDis (None, 44, 15, 15, 64)    18496     
_________________________________________________________________
time_distributed_19 (TimeDis (None, 44, 13, 13, 64)    36928     
__________

OSError: ignored

In [17]:
final_scores = []
for i in range(5):
  mod = load_model('/content/drive/My Drive/'+'conv_s_m'+str(i)+'.h5')
  scores = model.evaluate(X_test, y_test, verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  final_scores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(final_scores), np.std(final_scores)))

acc: 89.15%
acc: 89.15%
acc: 89.15%
acc: 89.15%
acc: 89.15%
89.15% (+/- 0.00%)
